In [1]:
import sqlite3
from fastapi import FastAPI
from pydantic import BaseModel

In [2]:
conn = sqlite3.connect("data.db")

cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS datos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT NOT NULL,
        edad INTEGER,
        curso TEXT NOT NULL
    )
""")

conn.commit()
conn.close()

In [3]:
class Datos(BaseModel):
    nombre: str
    edad: int
    curso: str

In [4]:
app = FastAPI()

@app.post("/agregar/")
async def agregar_datos(datos: Datos):
    conn = sqlite3.connect("data.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO datos (nombre, edad, curso) VALUES (?, ?, ?)", (datos.nombre, datos.edad, datos.curso))
    conn.commit()
    conn.close()
    return {"mensaje": "Datos agregados exitosamente"}

@app.get("/consultar/{id}/")
async def consultar_datos(id: int):
    conn = sqlite3.connect("data.db")
    cursor = conn.cursor()
    cursor.execute("SELECT nombre, edad, curso FROM datos WHERE id=?", (id,))
    resultado = cursor.fetchone()
    conn.close()
    if resultado is not None:
        return {"nombre": resultado[0], "edad": resultado[1], "curso": resultado[2]}
    else:
        return {"mensaje": "Datos no encontrados"}